# Crashes Near Point

Find the crash incident_id for a specific crash given its location and time. 

In [1]:
import folium
import datetime
import pandas as pd
from folium.plugins import HeatMap

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
# df = cda.crash_dataframe(verbose=True)

In [3]:
geocoded_location = 'Federal Blvd and W Jewell Ave, Denver, CO'

In [4]:
lat_lon = cda.geocode_location(geocoded_location)

# crash occurred within this distance of the given latitude and longitude
radius_miles = 0.25

query = f"""
select
*
, reported_date at time zone 'America/Denver' as reported_date_local
, ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
) as distance_feet

from crashes

where geo_lon is not null
and ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
    ) < {radius_miles * 5280}

order by reported_date
"""
# and extract(year from reported_date) >= 2022

nearby = pd.read_sql(query, cda.conn)

nearby['reported_date_local'] = nearby['reported_date_local'].dt.tz_localize('America/Denver')

In [5]:
nearby.groupby(nearby.reported_date.dt.year).agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int) #.to_clipboard()

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date,,,,,
2013,74,0,0,1,2
2014,99,1,1,0,0
2015,108,0,2,8,0
2016,114,1,1,2,1
2017,93,0,1,9,2
2018,101,0,3,6,0
2019,88,0,2,2,1
2020,80,0,3,0,0
2021,64,0,2,2,1


In [6]:
nearby['all_time'] = 'all time'
nearby.groupby('all_time').agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
all_time,,,,,
all time,974,3,19,38,7


In [7]:
display_df = nearby.copy()
display_df['bicycle'] = display_df['bicycle_ind'].replace(0, '').replace(1, True)
display_df['pedestrian'] = display_df['pedestrian_ind'].replace(0, '').replace(1, True)
display_df['serious_injury'] = display_df['sbi'].replace(False, '')
display_df['fatal'] = display_df['fatality'].replace(False, '')

display_cols = [
    'reported_date_local'
    , 'top_traffic_accident_offense'
    , 'incident_address_corrected'
    , 'bicycle'
    , 'pedestrian'
    , 'serious_injury'
    , 'fatal'
    , 'distance_feet'
    , 'incident_id'
]
# nearby.to_clipboard()
# nearby[display_cols]
display_df.loc[display_df.pedestrian_ind == 1, display_cols]

,reported_date_local,top_traffic_accident_offense,incident_address_corrected,bicycle,pedestrian,serious_injury,fatal,distance_feet,incident_id
47,2013-08-28 17:02:00-06:00,TRAF - ACCIDENT,2098 S FEDERAL BLVD,,True,,,1178.332316,2013409456
187,2015-02-06 16:02:00-07:00,TRAF - ACCIDENT,S FEDERAL BLVD / W COLORADO AVE,,True,,,986.686665,201572404
231,2015-07-07 17:35:00-06:00,TRAF - ACCIDENT - SBI,3000 W JEWELL AVE,,True,True,,51.750423,2015382061
233,2015-07-11 16:28:00-06:00,TRAF - ACCIDENT - HIT & RUN,S FEDERAL BLVD / W JEWELL AVE,,True,,,13.330907,2015390257
259,2015-10-12 11:59:00-06:00,TRAF - ACCIDENT,1940 S FEDERAL BLVD,,True,,,364.199265,2015595411
267,2015-10-30 19:43:00-06:00,TRAF - ACCIDENT - SBI,1800 BLOCK S FEDERAL BLVD,,True,True,,331.601270,2015636405
268,2015-10-31 23:51:00-06:00,TRAF - ACCIDENT - HIT & RUN,S HAZEL CT / W JEWELL AVE,,True,,,670.128292,2015638947
270,2015-11-06 17:08:00-07:00,TRAF - ACCIDENT,W JEWELL AVE / S CLAY ST,,True,,,1301.034718,2015651000
277,2015-12-11 19:47:00-07:00,TRAF - ACCIDENT,2061 S FEDERAL BLVD,,True,,,755.793985,2015720925
301,2016-02-29 13:13:00-07:00,TRAF - ACCIDENT,2069 S FEDERAL BLVD,,True,,,755.793985,2016129379


In [8]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(nearby[['geo_lat', 'geo_lon']]).add_to(this_map)
folium.Circle(location=lat_lon, weight=2, radius=radius_miles*1609.34).add_to(this_map)
this_map.fit_bounds(
    [nearby[['geo_lat', 'geo_lon']].min().values.tolist()
     , nearby[['geo_lat', 'geo_lon']].max().values.tolist()]
)
this_map

In [9]:
# Text string with frequency of crashes
num_years = 3

most_recent_timestamp = cda.most_recent_crash_timestamp()
start_timestamp = most_recent_timestamp - pd.DateOffset(years=num_years)
days_in_time_range = (most_recent_timestamp - start_timestamp).days

nearby_in_time_range = nearby[nearby.reported_date_local > start_timestamp].copy()

days_between_crashes = days_in_time_range / len(nearby_in_time_range)

print(
    f'The location "{geocoded_location}" has had {len(nearby_in_time_range)} traffic crashes'
    , f'within a {radius_miles} mile radius'
    , f'over the last {num_years} years, an average of {days_between_crashes:.1f} days between crashes.'
    , f'In that time, there were {nearby_in_time_range.fatality.sum()} deadly crashes'
    , f'and {nearby_in_time_range.sbi.sum()} crashes causing a serious bodily injury.'
    , f'There were {(nearby_in_time_range.pedestrian_ind > 0).sum()} crashes involving a pedestrian'
    , f'and {(nearby_in_time_range.bicycle_ind > 0).sum()} crashes involving a bicyclist.'
)

The location "Federal Blvd and W Jewell Ave, Denver, CO" has had 202 traffic crashes within a 0.25 mile radius over the last 3 years, an average of 5.4 days between crashes. In that time, there were 1 deadly crashes and 5 crashes causing a serious bodily injury. There were 9 crashes involving a pedestrian and 1 crashes involving a bicyclist.


In [10]:
cda.conn.dispose()